# 12: SalesOps Suite: Final Showcase 🎥

**"From Raw Data to Autonomous Action"**

This notebook visualizes the end-to-end journey of the SalesOps Agent.
It consumes the artifacts generated by our autonomous pipeline run.

### 📊 The Pipeline
1.  **Ingest:** 9,994 rows processed.
2.  **Detect:** Found statistical anomalies (Z-Score > 3.0).
3.  **Explain:** Gemini 2.0 analyzed root causes using RAG.
4.  **Act:** Created Jira tickets & Sent Emails.

## 1) Setup

In [1]:
import sys
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display, Markdown

# Add project root
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "..")))

from dashboard_data.loaders import (
    load_snapshot,
    load_anomalies,
    load_enriched,
    load_actions,
    get_kpis,
)

print("✅ Showcase Environment Ready")

✅ Showcase Environment Ready


## 2) KPIs

In [2]:
# 1. High-Level Business Health
df = load_snapshot()
kpis = get_kpis(df)

# Create a beautiful indicator chart
fig = go.Figure()

fig.add_trace(
    go.Indicator(
        mode="number+delta",
        value=kpis["revenue"],
        title={"text": "Total Revenue"},
        number={"prefix": "$"},
        domain={"row": 0, "column": 0},
    )
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=kpis["profit"],
        title={"text": "Total Profit"},
        number={"prefix": "$"},
        domain={"row": 0, "column": 1},
    )
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=kpis["margin"],
        title={"text": "Profit Margin"},
        number={"suffix": "%", "valueformat": ".1f"},
        domain={"row": 0, "column": 2},
    )
)

fig.update_layout(grid={"rows": 1, "columns": 3, "pattern": "independent"}, height=200)
fig.show()

## 3) The Problem - Anomalies

In [3]:
# 2. The Hidden Problems (Anomalies)
df_anoms = load_anomalies()

if not df_anoms.empty:
    # Filter for high severity
    top_anoms = df_anoms[df_anoms["score"] > 3.0].head(10)

    fig = px.scatter(
        top_anoms,
        x="period_start",
        y="score",
        size="score",
        color="detector",
        hover_data=["entity_id", "metric"],
        title="🚨 Critical Anomalies Detected (Z-Score > 3.0)",
        color_discrete_sequence=["#EF553B", "#636EFA"],
    )
    fig.show()

## 4) The Solution - AI Explanations 

In [4]:
# 3. The Intelligence (Gemini Explanations)
df_enriched = load_enriched()

if not df_enriched.empty:
    # Pick the most interesting one (Highest Score)
    best_rec = df_enriched.loc[df_enriched["score"].idxmax()]

    md = f"""
### 🤖 AI Insight Spotlight
**Anomaly:** {best_rec['entity_id']} ({best_rec['level']})

> **"{best_rec['explanation_short']}"**

* **Root Cause:** {best_rec['explanation_full']}
* **Confidence:** {best_rec['confidence']}
* **Action Proposed:** {best_rec['suggested_actions']}

*(Generated by {best_rec['model']} in {best_rec['latency']}ms)*
"""
    display(Markdown(md))


### 🤖 AI Insight Spotlight
**Anomaly:** South (region)

> **"Sales in the South region are significantly higher than expected, indicated by a score of 24.21."**

* **Root Cause:** The recorded sales for the South region, 23,661.23, are dramatically higher than the expected value of 1,159.38. This deviation is substantial, as the actual sales are over 20 times the expected amount. There are no historical anomalies to compare this against, making it a unique event.
* **Confidence:** High
* **Action Proposed:** ['Investigate the source of the reported sales figure to verify accuracy.', 'Analyze recent sales activities and deal closures within the South region for potential drivers.', 'Confirm if any new product launches or significant marketing campaigns occurred in the South region.']

*(Generated by gemini-2.5-flash-lite in 1389ms)*


## 5) The Action - Execution Log

In [5]:
# 4. Closing the Loop (Actions Taken)
df_actions = load_actions()

if not df_actions.empty:
    # Status Breakdown
    fig = px.pie(
        df_actions, names="type", title="Autonomous Actions Executed", hole=0.4
    )
    fig.show()

    print("Recent Audit Log:")
    display(df_actions[["timestamp", "type", "status"]].tail(3))

Recent Audit Log:


,timestamp,type,status
7,2025-11-27 17:18:28.359583+00:00,create_ticket,success
8,2025-11-27 17:18:30.398278+00:00,create_ticket,success
9,2025-11-27 17:18:32.439209+00:00,create_ticket,success


## 🏆 Mission Accomplished

We have demonstrated a fully autonomous SalesOps workflow.
* **Data:** Ingested & Monitored.
* **Insight:** Generated by AI with Memory.
* **Action:** Executed safely via APIs.
